In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics import classification_report

In [9]:
df = pd.read_csv(r"C:\Users\Deepak Thirukkumaran\OneDrive\Documents\Education\SNU b.tech A.I & D.S\2nd year_2022-23\4th SEMESTER\DAA\CIA_2\Bank_Personal_Loan_Modelling.csv")

In [10]:
print("Shape : ", df.shape)
df.head()

Shape :  (5000, 14)


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [11]:
df.drop(["ID", "ZIP Code"],axis=1,inplace=True)
print("Duplicate : ", df.duplicated().sum())
print("Null : ", df.isnull().sum().sum())

Duplicate :  13
Null :  0


In [12]:
print(df["Experience"].unique())
df["Experience"] = abs(df["Experience"])

[ 1 19 15  9  8 13 27 24 10 39  5 23 32 41 30 14 18 21 28 31 11 16 20 35
  6 25  7 12 26 37 17  2 36 29  3 22 -1 34  0 38 40 33  4 -2 42 -3 43]


In [13]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [14]:
df = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg','Education', 'Mortgage', 'Securities Account','CD Account', 'Online', 'CreditCard', 'Personal Loan']]


In [15]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [16]:
x,y = X,Y

In [22]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=53)

In [23]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [24]:
x_train.shape, y_train.shape

((3500, 11), (3500,))

In [25]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [26]:
BATCH_SIZE = 3

In [27]:
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)

In [28]:
train_x.shape, train_y.shape

(torch.Size([3500, 11]), torch.Size([3500]))

In [29]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

In [30]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [31]:
model = Model()
print(model)

Model(
  (layer1): Linear(in_features=11, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


In [41]:
from pyswarms.single import GlobalBestPSO
torch.manual_seed(420)
torch.set_grad_enabled(False)

model = Model()
par = np.concatenate([i.numpy().flatten() for i in model.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]

print("Dim : ", len(par))
print("Layers Shape : ", shape)
print("Layers Size : ", size)

Dim :  209
Layers Shape :  [(16, 11), (16,), (1, 16), (1,)]
Layers Size :  [176, 16, 16, 1]


In [34]:
def objective_function(particle,shape=shape,size=size):
    accuracy = []
    output = []
    

    for par in particle:
        param = list()
        cum_sum = 0
        for i in range(len(size)):
            array = par[cum_sum : cum_sum + size[i]]
            array = array.reshape(shape[i])
            cum_sum += size[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    for i in range(len(output)):
 
        model = Model()
        for idx,wei in enumerate(model.parameters()):
            wei.data = (torch.tensor(output[i][idx])).to(torch.float32)
        
    
        y_pred = model(train_x)
        y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
        acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
        accuracy.append(1 - acc) 
        
    return accuracy

In [36]:
options = {'c1': 0.7, 'c2': 0.1, 'w': 0.3}
dim = len(param)
x_max = 1.0 * np.ones(dim)
x_min = -1.0 * x_max
bounds = (x_min, x_max)

pso = GlobalBestPSO(n_particles=500, dimensions=209, options=options, bounds=bounds)

In [37]:
best_cost, best_parameters = pso.optimize(objective_function, iters=50)
print("Accuracy : ", 1 - best_cost)

2023-04-05 13:46:23,335 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.7, 'c2': 0.1, 'w': 0.3}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████████|50/50, best_cost=0.0377
2023-04-05 13:46:35,537 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0377142857142857, best pos: [-1.35679324e-01  3.16049739e-01 -1.23684229e-01  1.20945646e-01
  3.64310680e-01  6.10649313e-02  3.06468987e-01 -1.78698614e-01
  1.62835209e-01 -1.26261108e-01  9.72018351e-02  8.45059575e-02
 -8.94865164e-02  4.36738295e-02 -2.21200933e-01  3.00675210e-01
 -3.36064257e-01  8.74037586e-02  9.45288183e-02 -7.78877263e-02
 -2.54018333e-02  1.22610051e-01  5.71812284e-02  4.24324307e-01
 -9.11532662e-02  5.55955462e-03  2.10131160e-01  8.19831360e-02
  3.59814844e-01  9.88443028e-02 -5.80634063e-02  1.14470657e-01
 -2.58700378e-01 -1.45230865e-02  3.65495518e-01 -2.10330482e-01
 -2.19853631e-01  1.85184400e-01  3.33075277

Accuracy :  0.9622857142857143


In [38]:
result = []
for par in [best_parameters]:
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = par[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    result.append(param)

best_model = Model()
for idx,wei in enumerate(best_model.parameters()):
    wei.data = (torch.tensor(result[0][idx])).to(torch.float32)


y_pred = best_model(train_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
print("Accuracy : ", acc)

Accuracy :  0.9622857142857143


In [39]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

In [40]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1390
           1       0.66      0.89      0.76       110

    accuracy                           0.96      1500
   macro avg       0.83      0.93      0.87      1500
weighted avg       0.97      0.96      0.96      1500

